In [2]:
import ezdxf
import numpy as np
import xml.etree.ElementTree as ET

In [3]:

# 打开 CAD 文件
doc = ezdxf.readfile("../data/和县/现状条件.dxf")

# 获取模型空间（model space）
msp = doc.modelspace()


In [ ]:
count = 0
for entity in msp:
    if entity.dxf.layer == 'DX-地形':
        if entity.dxftype() == 'INSERT':  # 判断实体类型为 INSERT
            entity.explode()  # 对 INSERT 实体进行分解
for entity in msp:
    if entity.dxf.layer == 'DX-地形':
        count += 1
count

In [30]:
# 指定要提取的图层名称

road_layer_mapper = {
    '车行-主干道':0,
    '车行-支路':1,
    '车行-次干道':1,
    '车行-街巷':2,
    '人行-街巷':3
}
height_layer = '高程点'

data = {'roads':[], 'buildings':[], 'height':[]}

for entity in msp:

    if entity.dxf.layer in road_layer_mapper.keys():
        if entity.dxftype() == 'LINE' or entity.dxftype() == 'LWPOLYLINE':
            points = np.array(entity.get_points())
            points = points[:, :2].tolist()
            road_data = {
                'points':points,
                'level':road_layer_mapper[entity.dxf.layer],
                'state':0
            }
            data['roads'].append(road_data)
    elif entity.dxf.layer == height_layer:
        if entity.dxftype() == 'TEXT':  # 判断实体类型为文本
            # text_content = entity.dxf.text  # 获取文字内容
            insertion_point = entity.dxf.insert
            data['height'].append(insertion_point.xyz)




In [31]:
# 将字典转换为 XML
def dict_to_xml(dictionary, parent):
    for key, value in dictionary.items():
        if isinstance(value, dict):
            dict_to_xml(value, ET.SubElement(parent, key))
        else:
            ET.SubElement(parent, key).text = str(value)

root = ET.Element("data")
dict_to_xml(data, root)

# 创建 ElementTree 对象
tree = ET.ElementTree(root)

# 保存为 XML 文件
tree.write("output.xml", encoding="utf-8", xml_declaration=True)

In [19]:
# 从 XML 文件中读取数据
tree = ET.parse('output.xml')
root = tree.getroot()

# 将 XML 解析为字典
def xml_to_dict(element):
    dictionary = {}
    for child in element:
        if child:
            value = xml_to_dict(child)
        else:
            value = child.text
        if child.tag in dictionary:
            if type(dictionary[child.tag]) is list:
                dictionary[child.tag].append(value)
            else:
                dictionary[child.tag] = [dictionary[child.tag], value]
        else:
            dictionary[child.tag] = value
    return dictionary

data = xml_to_dict(root)
print(data)

{'roads': "[{'points': [[500552.3221342148, 3511831.145854538], [500571.095, 3511816.273], [500630.6940943382, 3511784.645891888], [500667.1406372773, 3511770.827647552], [500710.1306276421, 3511758.235568561], [500749.0922782491, 3511750.247442778], [500809.0179999999, 3511747.16], [500860.8026737844, 3511749.963402415], [500918.7480000001, 3511764.193], [500961.9948087768, 3511779.102359301], [500994.1470000001, 3511793.731], [501095.114, 3511837.973], [501201.879, 3511884.769], [501285.449, 3511921.477], [501396.6790000001, 3511970.249], [501461.631, 3511998.697], [501534.183, 3512030.557], [501646.174, 3512079.486000001], [501729.124, 3512115.061000001], [501769.816506037, 3512129.726924122], [501801.042, 3512139.944000001], [501874.634, 3512159.449000001], [501950.995, 3512178.36], [502024.194, 3512195.923], [502077.178, 3512209.03], [502166.294, 3512230.824000001], [502221.9921721484, 3512244.435869033], [502289.539, 3512260.918000001], [502346.1730000001, 3512274.714000001], [50